In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore('./.cache/')

# Token 방식으로 텍스트를 인식
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",                 # 특정 기준으로 분할
    chunk_size=600,
    chunk_overlap=100,
)

# UnstruturedFileLoader : 많은 파일들을 불러올 수 있음. docx, xlsx, pdf, ppt, txt 등 많은 파일들을 불러올 수 있는 Loader
loader= UnstructuredFileLoader("./files/대입합불정리_세특포함_디랩.pdf")

docs = loader.load_and_split(text_splitter=splitter)                                                                                                                                           


embeddings = OpenAIEmbeddings()

cacehd_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# 캐시를 적용하여 임베딩
# vectorstore = Chroma.from_documents(docs, cacehd_embeddings)
vectorstore = FAISS.from_documents(docs, cacehd_embeddings)


chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever()
)

chain.run("성균관대를 합격자의 평균 내신등급은 어느 정도야 ?")

TypeError: unsupported operand type(s) for +=: 'OpenAIObject' and 'OpenAIObject'